# Fancy Decorators

In the second part of this tutorial, we explore more advanced
features including:

- Add **decoractors to classes**
- Add **several decorators** to one function
- Create decorators with **arguments**
- Create decorators that can **optionally** take arguments
- Define **stateful** decorators
- **Define classes** that act as decorators

## Decorating Classes

One can use decorators on classes in two different ways

- **Decorate** the **methods of a class**
- Decorate the **whole** class

Some decorations for the methods of a class are even built-ins in
Python, including:

- `@classmethod`
- `@staticmethod`
- `@property`


For example, we can decorate methods using the `@trace` and `@timer`
decorators defined earlier.

In [1]:
from secrets import token_hex
from xxsubtype import bench

from decorators import trace, timer

class TimeWaster:
    @trace
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([number ** 2 for number in range(self.max_num)])

Using this class, one can see the effect of the decorators.

In [2]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x1063e0590>, 1000)
__init__() returns None


In [3]:
tw.waste_time(999)

Finished 'waste_time' in 0.0368 seconds


The other way to use decorators is to **decorate the whole class**.

This decoration of the entire class is done, for example, in the
`dataclasses` module.

In [4]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

The meaning of the syntax is similar to that of a function decorator.
For exaple, one could have decorated the `PlayingCard` class by the
expression, `PlayingCard = dataclass(PlayingCard)`.

A common use case of class decorators is to be a simpler alternative
to some use cases of **metaclasses**. In both cases, one is changing
the definition of a class **dynamically**.

Writing a class decorator is similar to writing a function decorator.
The only difference is that the decorator will receive a **class**
and not a function as an arguments. In fact, all the decorators in
our previous notebooks **will** work as class decorators.

However, when you're using these decorators on a class method instead
of a function, their effect **might not be what you want.

For example, let's apply the `@timer` decorator to a class.

In [5]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i ** 2 for i in range(self.max_num)])

Decorating a class **doesn't** decorate its methods. Instead, `@timer`
only measures the time it takes to instantiate the class.

In [6]:
tw = TimeWaster(1000)

Finished 'TimeWaster' in 0.0000 seconds


In [7]:
tw.waste_time(999)

Note that the output of `@timer` is only printed when the instance,
`tw`, is created. The call to `waste_time()` **is not timed**.

Later, we'll see an example of defining a proper class decorator,
namely, `@singleton`, which ensures that only one instance of
class exists at runtime.

## Nesting Decorators

One can _apply several decorators_ to a function at once by stacking
them on top of each other.

In [8]:
from decorators import trace, do_twice

@trace
@do_twice
def greet(name):
    print(f'Hello {name}')

Remember that applying two decorators as we do previously results in

- `@trace calls @do_twice which calls `greet()`

which is the same as `trace(do_twice(greet()))

In [9]:
greet('Yadi')

Calling greet('Yadi')
Hello Yadi
Hello Yadi
greet() returns None


The greeting is printed twice because of `@do_twice`. However, the
output from `@trace` is only shown **once** since the call to `trace`
occurs **before** the `@do_twice` decorator.

In [10]:
@do_twice
@trace
def greet(name):
    print(f'Hello {name}')

greet('Yadi')

Calling greet('Yadi')
Hello Yadi
greet() returns None
Calling greet('Yadi')
Hello Yadi
greet() returns None


Here, `@do_twice` is applied to @trace. Consequently, one can see that
both calls to `greet()` are annotated with trace information.

## Defining Decorators with Arguments

Sometimes, it is useful to _pass arguments to your decorators_. For
instance, `@do_twice` could be extended to a `@repeat(num_times)`
decorator. In this situation, the number of repetitions could be
given as an argument.

In [11]:
from decorators import repeat

There are a few subtle things happening in the `repeat()` function:

- Defining `decorator_repeat()` as an inner function means that
  `repeat()` refers to a function object, `decorator_repeat`. Earlier,
  you used decorators like `@do_twice` **without** parentheses. Now,
  you **must** add parentheses when setting up the decorator, as in
  `@repeat()`. This is required to **add arguments**.
- The `num_times` argument is seemingly not used in `repeat()` itself.
  But by passing `num_times` to `repeat()`, a **closure** is created
  where the value of `num_times` is stored until `wrapper_repeat()`
  uses it later.

Here are our results:

In [12]:
@repeat(num_times=4)
def greet(name):
    print('Hello', name)

greet('World)')

Hello World)
Hello World)
Hello World)
Hello World)


## Creating Decorators with Optional Arguments

With a little bit of care, you can also define _decorators that can
be used both with and without arguments.

When a decorator uses arguments, you need to add an extra outer function.
The challenge now is for your code to figure out if you've called the
decorator with or without arguments. This means that the decorator
arguments must all be specified by keyword. You can enforce this with the
special asterisk (`*`) syntax, which means that **all the following
parameters are keyword-only**.

For example,

```python
def name(_func=None, *, key1=value1, key2=value2, ...):
    def decorator_name(func):
        # ... # Create and return a wrapper function.

    if _func is None:
        return decorator_name
    else:
        return decorator_name(_func)
```
